# Setup Model

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from UtilityMethods import utils
import sys
import pickle
import time
import pulp as p
import math
from copy import copy
import pprint as pp
import itertools
from tqdm import tqdm

## Global variables

In [27]:
# Global variables

IS_VISIT_DEPENDENT = False # whether the above empirical estimates are visit-dependent or not
# DATA = '../data/ACCORD_BPClass_v2.csv' # raw data, the state features' levels are not merged
# DATA2 = '../data/ACCORD_BPClass_v2_merged.csv' # _merged means sbp_discrete levels are merged, _contextual means we added baseline_BMI_discrete, but we are not using it
# DATA3= '../data/ACCORD_BGClass_v2_merged_Contextual.csv'

DATA3= '../data/ACCORD_BGClass_v2_merged.csv'

EPISODE_LENGTH = 20 # average number of visits per patient
# CONSTRAINT_list = [16, 10, 10] # deviation * 20 visits 
# C_b_list = [8, 5, 5]  # change this if you want different baseline policy.

CONSTRAINT_list = [16, 16, 16] # deviation * 20 visits 
C_b_list = [8, 8, 8]  # change this if you want different baseline policy.

delta = 0.01 # bound


EPS = 0.01 # not used
M = 0 # not used

## Preprocess the data

In [28]:
df = pd.read_csv(DATA3)
print(df.columns)

Index(['MaskID', 'Visit', 'glycemia', 'bp', 'sbp', 'dbp', 'hr', 'hba1c', 'TC',
       'trig', 'vldl', 'ldl', 'hdl', 'fpg', 'alt', 'cpk', 'potassium',
       'screat', 'gfr', 'ualb', 'ucreat', 'uacr', 'edu_baseline',
       'yrsdiab_baseline', 'yrstens_baseline', 'cigarett_baseline',
       'wt_kg_baseline', 'ht_cm_baseline', 'wt_kg_visit', 'ht_cm_visit',
       'oral_gmed', 'medadd', 'medchg_intbp', 'medchg_stdbp', 'bp_med', 'BMI',
       'female', 'baseline_age', 'cvd_hx_baseline', 'raceclass', 'type_po',
       'CVDRisk', 'BPClass', 'BGClass', 'sbp_discrete', 'hba1c_discrete',
       'BMI_discrete', 'hdl_discrete', 'TC_discrete', 'sbp_feedback',
       'hba1c_feedback', 'CVDRisk_feedback', 'bgclass_none', 'Bingu', 'Thiaz',
       'Sulfon', 'Meglit', 'Alpha-gluc', 'baseline_BMI', 'race_whiteother',
       'race_black', 'CVDRisk_feedback_binary', 'BMI_feedback', 'TC_feedback',
       'hba1c_discrete_merged', 'action_code', 'state_code', 'edu_baseline_1',
       'edu_baseline_2', 'edu_b

In [29]:
# check if any BMI value is <0 or >100
print(df[df['BMI']<0])
print(df[df['BMI']>100])
print(df.shape)

Empty DataFrame
Columns: [MaskID, Visit, glycemia, bp, sbp, dbp, hr, hba1c, TC, trig, vldl, ldl, hdl, fpg, alt, cpk, potassium, screat, gfr, ualb, ucreat, uacr, edu_baseline, yrsdiab_baseline, yrstens_baseline, cigarett_baseline, wt_kg_baseline, ht_cm_baseline, wt_kg_visit, ht_cm_visit, oral_gmed, medadd, medchg_intbp, medchg_stdbp, bp_med, BMI, female, baseline_age, cvd_hx_baseline, raceclass, type_po, CVDRisk, BPClass, BGClass, sbp_discrete, hba1c_discrete, BMI_discrete, hdl_discrete, TC_discrete, sbp_feedback, hba1c_feedback, CVDRisk_feedback, bgclass_none, Bingu, Thiaz, Sulfon, Meglit, Alpha-gluc, baseline_BMI, race_whiteother, race_black, CVDRisk_feedback_binary, BMI_feedback, TC_feedback, hba1c_discrete_merged, action_code, state_code, edu_baseline_1, edu_baseline_2, edu_baseline_3, cigarett_baseline_1]
Index: []

[0 rows x 71 columns]
Empty DataFrame
Columns: [MaskID, Visit, glycemia, bp, sbp, dbp, hr, hba1c, TC, trig, vldl, ldl, hdl, fpg, alt, cpk, potassium, screat, gfr, ualb,

### Categorize the context features

In [30]:
# catrgorize the edu_baseline into 3 columns 

edu1=[]
edu2=[]
edu3=[]
for i in range(df.shape[0]):
    row = df.iloc[i]
    edu = row['edu_baseline']      

    if edu == 1:
        edu1.append(1)
        edu2.append(0)
        edu3.append(0)
    elif edu == 2:
        edu1.append(0)
        edu2.append(1)
        edu3.append(0)
    elif edu == 3:
        edu1.append(0)
        edu2.append(0)
        edu3.append(1)
    elif edu == 4:
        edu1.append(0)
        edu2.append(0)
        edu3.append(0)
    else:
        print('error')
        exit()

df['edu_baseline_1'] = edu1
df['edu_baseline_2'] = edu2
df['edu_baseline_3'] = edu3

In [31]:
# categorize the cigarett_baseline into 0/1, 1  = 1, 2 = 0
cig1 = []
for i in range(df.shape[0]):
    row = df.iloc[i]
    cig = row['cigarett_baseline']
    if cig == 1:
        cig1.append(1)
    elif cig == 2:
        cig1.append(0)
    else:
        print('error')
        exit()

df['cigarett_baseline_1'] = cig1

In [32]:
# categorize the sbp_discrete_merged (0, 1, 2) into 2 columns, BUT WE WILL NOT USE 2 columns, but just 1 column
# sbp1 = []
# sbp2 = []
# for i in range(df.shape[0]):
#     row = df.iloc[i]
#     sbp = row['sbp_discrete_merged']
#     if sbp == 0:
#         sbp1.append(1)
#         sbp2.append(0)
#     elif sbp == 1:
#         sbp1.append(0)
#         sbp2.append(1)
#     elif sbp == 2:
#         sbp1.append(0)
#         sbp2.append(0)
#     else:
#         print('error')
#         exit()

# df['sbp_discrete_merged_1'] = sbp1
# df['sbp_discrete_merged_2'] = sbp2

In [33]:
# overwrite to csv
df.to_csv(DATA3, index=False)
print(df.shape)
print(df.columns)

(290467, 71)
Index(['MaskID', 'Visit', 'glycemia', 'bp', 'sbp', 'dbp', 'hr', 'hba1c', 'TC',
       'trig', 'vldl', 'ldl', 'hdl', 'fpg', 'alt', 'cpk', 'potassium',
       'screat', 'gfr', 'ualb', 'ucreat', 'uacr', 'edu_baseline',
       'yrsdiab_baseline', 'yrstens_baseline', 'cigarett_baseline',
       'wt_kg_baseline', 'ht_cm_baseline', 'wt_kg_visit', 'ht_cm_visit',
       'oral_gmed', 'medadd', 'medchg_intbp', 'medchg_stdbp', 'bp_med', 'BMI',
       'female', 'baseline_age', 'cvd_hx_baseline', 'raceclass', 'type_po',
       'CVDRisk', 'BPClass', 'BGClass', 'sbp_discrete', 'hba1c_discrete',
       'BMI_discrete', 'hdl_discrete', 'TC_discrete', 'sbp_feedback',
       'hba1c_feedback', 'CVDRisk_feedback', 'bgclass_none', 'Bingu', 'Thiaz',
       'Sulfon', 'Meglit', 'Alpha-gluc', 'baseline_BMI', 'race_whiteother',
       'race_black', 'CVDRisk_feedback_binary', 'BMI_feedback', 'TC_feedback',
       'hba1c_discrete_merged', 'action_code', 'state_code', 'edu_baseline_1',
       'edu_baseli

## State space and action space

In [34]:
# state space, actions available in each state are always the same

"""
state_features = ['sbp_discrete','hba1c_discrete','TC_discrete','hdl_discrete','BMI_discrete'] 
sbp_level = ['0', '1', '2', '3'] # possible values for sbp_discrete
hba1c_level = ['0', '1', '2', '3', '4', '5', '6', '7']
TC_level = ['0', '1', '2', '3']
hdl_level = ['0', '1', '2', '3']
BMI_level = ['0', '1', '2', '3']
"""

# here we merge levels
sbp_level = ['0', '1', '2'] # sbp_discrete, 0: 0, 1:1, 2+3: 2
hba1c_level = ['0', '1', '2'] # hba1c_discrete, 0+1: 0, 2+3: 1, 4+5+6+7: 2
TC_level = ['0', '1'] # TC_discrete, 0+1: 0, 2+3: 1
hdl_level = ['0', '1'] # hdl_discrete, 0+1: 0, 2+3: 1

sbp_discrete_code_dict = {'0': '0', '1': '1',
                          '2': '2', '3': '2',}

hba1c_discrete_code_dict = {'0': '0', '1': '0', 
                            '2': '1', '3': '1', 
                            '4': '2', '5': '2', 
                            '6': '2', '7': '2'}

TC_discrete_code_dict = {'0': '0', '1': '0',
                         '2': '1', '3': '1'}

hdl_discrete_code_dict = {'0': '0', '1': '0',
                          '2': '1', '3': '1'}

# 4 features, state space = 36
# state_features = ['sbp_discrete', 'hba1c_discrete', 'TC_discrete', 'hdl_discrete'] 
# combinations = itertools.product(sbp_level, hba1c_level, TC_level, hdl_level)

# 3 features, state space = 18
# state_features = ['sbp_discrete', 'hba1c_discrete', 'TC_discrete'] 
# combinations = itertools.product(sbp_level, hba1c_level, TC_level)

# 2 features, state space = 9
# combinations = itertools.product(sbp_level, hba1c_level)
# state_features = ['sbp_discrete', 'hba1c_discrete'] 

# 1 feature, srtate space = 3
combinations = itertools.product(hba1c_level)
state_features = ['hba1c_discrete'] 

states = [''.join(i) for i in combinations]
print('len(states) =', len(states))
print(states[:5])

N_STATES = len(states) 
state_code_to_index = {code: i for i, code in enumerate(states)}
state_index_to_code = {i: code for i, code in enumerate(states)}
# print the first 5 state_code_to_index
for i in range(3):
    print(states[i], state_code_to_index[states[i]])
print()



# action space, 000000000 means bgclass_none, 111111111 means all bgmed class are precribed
# we donot include 'bgclass_none' as a action, because 000000000 means bgclass_none
action_features = ['Bingu', 'Thiaz', 'Sulfon', 'Meglit'] # pick the top 4 most frequently prescribed bgmed class 

combinations = list(itertools.product('01', repeat=len(action_features)))
actions = [''.join(i) for i in combinations]
print('len(actions) =', len(actions))
N_ACTIONS = len(actions) # number of actions = 512
action_code_to_index = {code: i for i, code in enumerate(actions)}
action_index_to_code = {i: code for i, code in enumerate(actions)}
# print the first 5 action_code_to_index
for i in range(5):
    print(actions[i], action_code_to_index[actions[i]])

# build the action space for each state, assign the same action space to all states
ACTIONS_PER_STATE = {}
for s in range(N_STATES):
    ACTIONS_PER_STATE[s] = [i for i in range(N_ACTIONS)] # this is the action code index
print('Actions for State 0:', ACTIONS_PER_STATE[0])

len(states) = 3
['0', '1', '2']
0 0
1 1
2 2

len(actions) = 16
0000 0
0001 1
0010 2
0011 3
0100 4
Actions for State 0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


## Calculate empirical estimates of P

In [35]:
df = pd.read_csv(DATA3)

# save df to csv, replace _merged.csv with _contextual.csv
fn = '../data/ACCORD_BGClass_v2_contextual.csv'
df.to_csv(fn, index=False)

In [36]:
# add the state and action code columns
action_code = []
state_code = []
hba1c_discrete_merged = []
for i in range(len(df)):
    row = df.iloc[i]
    s_code = ''
    a_code = ''
    for state_fea in state_features:
        code = str(row[state_fea])

        # merge 3 with 2 for sbp_discrete and TC_discrete
        if state_fea == 'sbp_discrete':
            code = sbp_discrete_code_dict[code]            
        elif state_fea == 'hba1c_discrete':
            code = hba1c_discrete_code_dict[code]
            hba1c_discrete_merged.append(code)
        elif state_fea == 'TC_discrete':
            code = TC_discrete_code_dict[code]
        elif state_fea == 'hdl_discrete':
            code = hdl_discrete_code_dict[code]
        else:
            raise ValueError('state_fea not recognized')
            exit(1)       
        
        s_code += code
    
    for action_fea in action_features:
        a_code += str(row[action_fea])
    
    action_code.append(a_code)
    state_code.append(s_code)

assert len(hba1c_discrete_merged) == len(df)

df['hba1c_discrete_merged'] = hba1c_discrete_merged
df['action_code'] = action_code
df['state_code'] = state_code
print('Finished adding action_code and state_code columns')

# DATA_MERGED = DATA[:-4] + '_merged.csv'
# # write the merged data to file
# df.to_csv(DATA_MERGED, index=False)

Finished adding action_code and state_code columns


In [37]:
#------------- calculate the empirical estimate of P based on entire dataset ----------------
        
count_s_a = {} # count the number of times state s and action a appear in the dataset, sparse format
count_s_a_d = {} # count the number of times state s, action a, and next state s' appear in the dataset
visit_number = [] # number of visits for each patient

# loop through each patient in the dataset
for i in tqdm(range(100001, 110252)):
    df_patient = df[df['MaskID'] == i]

    if len(df_patient) > 0:
        visit_number.append(len(df_patient))

    # loop through each visit of the patient
    for j in range(len(df_patient)-1): # loop before last visit
        row = df_patient.iloc[j]
        s_code = row['state_code']
        a_code = row['action_code']
        ns_code = df_patient.iloc[j+1]['state_code']

        # convert from code to index
        s = state_code_to_index[s_code]
        a = action_code_to_index[a_code]
        s_ = state_code_to_index[ns_code]

        if (s, a) not in count_s_a:
            count_s_a[(s, a)] = 1
        else:
            count_s_a[(s, a)] += 1

        if (s, a, s_) not in count_s_a_d:
            count_s_a_d[(s, a, s_)] = 1
        else:
            count_s_a_d[(s, a, s_)] += 1

print('len(visit_number) =', len(visit_number))
print('averge visit_number =', sum(visit_number)/len(visit_number))

print('len(count_s_a) =', len(count_s_a))
print('len(count_s_a_d) =', len(count_s_a_d))
print('Finished counting by looping through the dataset')

100%|██████████| 10251/10251 [01:27<00:00, 116.73it/s]

len(visit_number) = 8075
averge visit_number = 35.97114551083591
len(count_s_a) = 48
len(count_s_a_d) = 144
Finished counting by looping through the dataset


In [38]:
# calculate the sparsity of state-action pairs
print('Total possible state-action pairs =', N_STATES * N_ACTIONS)
print('Seen state-action pairs =', len(count_s_a))
print('Unseen state-action pairs =', N_STATES * N_ACTIONS - len(count_s_a))
print('Sparsity of state-action pairs =', 1 - len(count_s_a)/(N_STATES * N_ACTIONS))

Total possible state-action pairs = 48
Seen state-action pairs = 48
Unseen state-action pairs = 0
Sparsity of state-action pairs = 0.0


In [39]:
# initialize P, NOT using sparse matrix format
P = {} # N_STATES * N_ACTIONS * N_STATES, dictionary of transition probability matrices, based on the entire dataset

for s in range(N_STATES):
    l = len(actions)

    P[s] = {}    
    for a in range(N_ACTIONS):
        P[s][a] = np.zeros(N_STATES)
        
print('Finished initializing P')

for (s, a, s_) in count_s_a_d:
    P[s][a][s_] = count_s_a_d[(s, a, s_)]/max(count_s_a[(s, a)],1)

print('Finished calculating the empirical estimate of P')

#------------- check the sparsity of P, R, C
print('\nDetails of P, R, C:')
print('P: {:.6f}% are non-zeros'.format(len(count_s_a_d)*100/(N_STATES*N_ACTIONS*N_STATES)))

# print sample values of P, R, C
print('\nSample values of P, R, C:')
print('P[0][0][0] =', P[0][0][0])

Finished initializing P
Finished calculating the empirical estimate of P

Details of P, R, C:
P: 100.000000% are non-zeros

Sample values of P, R, C:
P[0][0][0] = 0.9334208552138035


## Check Init states

In [40]:
def check_frequency(df, col_name):
    print(col_name)
    df = df[col_name]
    df = df.value_counts()
    print(df)
    print()

    # return the first index in the series
    return df.index[0]
    
# get the rows when the visit=='BLR' in df
df_blr = df[df['Visit']=='BLR']
INIT_STATES_LIST = df_blr['state_code'].unique() # we will sample uniformly from this list
print('len(INIT_STATES_LIST) =', len(INIT_STATES_LIST))

print('df_blr.shape =', df_blr.shape)
most_freq_blr_state = check_frequency(df_blr, 'state_code')
print('most_freq_blr_state =', most_freq_blr_state)

INIT_STATE_INDEX = state_code_to_index[most_freq_blr_state]
print('INIT_STATE_INDEX =', INIT_STATE_INDEX)

len(INIT_STATES_LIST) = 3
df_blr.shape = (8075, 71)
state_code
2    6414
1    1564
0      97
Name: state_code, dtype: int64

most_freq_blr_state = 2
INIT_STATE_INDEX = 2


## Check frequency of context features

In [41]:
context_fea = ['baseline_age', 'female', 'race_whiteother',
                'edu_baseline_1',
                'edu_baseline_2',
                'edu_baseline_3',
                'cvd_hx_baseline', 
                'baseline_BMI', 
                # 'baseline_BMI_discrete',
                # 'cigarett_baseline',
                'cigarett_baseline_1',
               ]

In [42]:
def get_context_vec(row, context_fea):
    context_vec = np.zeros(len(context_fea))
    for i in range(len(context_fea)):
        context_vec[i] = row[context_fea[i]]
    return context_vec 

def get_value_recorded(df, mask_id, col_name):
    df_patient = df[df['MaskID'] == mask_id]
    if len(df_patient) > 0:
        return df_patient[col_name]
    else:
        print('error: mask_id not found')
        return []


# build the CONTEXT_VECTOR_dict, each key is the MaskID, value is the corresponding CONTEXT_VECTOR for the patient
CONTEXT_VECTOR_dict = {}

# get unique MaskID, 4366 patients
mask_id_list = df['MaskID'].unique()
print('len(mask_id_list) =', len(mask_id_list))

# get the BLR visit only
df_blr = df[df['Visit']=='BLR']
print('df_blr.shape =', df_blr.shape)

# loop through each row of df_blr
for r in range(df_blr.shape[0]):
    row = df_blr.iloc[r]
    mask_id = row['MaskID']
    # state_code = row['state_code']
    context_vec = get_context_vec(row, context_fea)

    state_recorded = get_value_recorded(df, mask_id, 'state_code')
    action_recorded = get_value_recorded(df, mask_id, 'action_code')

    # for each state in state_recorded, convert it to state_index
    state_index_recorded = [state_code_to_index[s] for s in state_recorded]
    action_index_recorded = [action_code_to_index[a] for a in action_recorded]
    init_state_index = state_index_recorded[0]

    CONTEXT_VECTOR_dict[mask_id] = (context_vec, init_state_index, state_index_recorded, action_index_recorded)

print('len(CONTEXT_VECTOR_dict) =', len(CONTEXT_VECTOR_dict))


len(mask_id_list) = 8075
df_blr.shape = (8075, 71)
len(CONTEXT_VECTOR_dict) = 8075


In [43]:
print(df.shape)

(290467, 71)


## Build the KNN model for selecting clinician' action

1. build the KNN model using (context_fea, state_index) in the raw recorded data
2. save the fitted KNN model for use in Contextual_test.py

In [44]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

# Sample data - list of vectors representing points
# use the following columns of df to build the points, context_fea = ['baseline_age', 'female', 'race_whiteother', 'edu_baseline_1', 'edu_baseline_2', 'edu_baseline_3', 'cvd_hx_baseline', 'baseline_BMI', 'cigarett_baseline_1']

df['state_index'] = [state_code_to_index[s] for s in df['state_code']]
fea_cols = context_fea + ['state_index']
points = df[fea_cols].values.tolist()
print('points[0] =', points[0])
print('len(points) =', len(points))

# Corresponding labels for each point
labels = [action_code_to_index[a] for a in df['action_code']]
print('labels =', labels)
print('len(labels) =', len(labels))


# Vector point for which we want to find the nearest neighbors, sample point
vector_point = [60.8,	0,	1,	0,	0,	0,	0,	35.91217711,	0,	1]

# Create MinMaxScaler and fit_transform the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_points = scaler.fit_transform(points)
normalized_vector_point = scaler.transform([vector_point])

# Number of nearest neighbors to find
k = 5

# Create NearestNeighbors model and fit the dataset
knn = NearestNeighbors(n_neighbors=k)
knn.fit(normalized_points)

# Find the indices of k-nearest neighbors
distances, indices = knn.kneighbors(normalized_vector_point)
print('distances =', distances)
print('indices =', indices)

# Get the labels of the k-nearest neighbors
nearest_labels = [labels[i] for i in indices[0]]
print('nearest_labels =', nearest_labels)

# save the knn model to pickle file
with open('output_final2/knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

with open('output_final2/knn_model_label.pkl', 'wb') as f:
    pickle.dump(labels, f)

# save the scaler model to pickle file
with open('output_final2/scaler_model.pkl', 'wb') as f:
    pickle.dump(scaler, f)


points[0] = [60.8, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 35.91217711238743, 0.0, 2.0]
len(points) = 290467
labels = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 0, 2, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 10, 10, 10, 10, 14, 15, 13, 13

### Save the model settings

In [45]:
# dump the model settings and parameters to a pickle file
CONTEXT_VEC_LENGTH = len(context_fea)
ACTION_CODE_LENGTH = len(action_index_to_code[0])
print('CONTEXT_VEC_LENGTH =', CONTEXT_VEC_LENGTH)
print('ACTION_CODE_LENGTH =', ACTION_CODE_LENGTH)

with open('output_final2/model_contextual_BG.pkl', 'wb') as f:
    pickle.dump([P, CONTEXT_VEC_LENGTH, ACTION_CODE_LENGTH, CONTEXT_VECTOR_dict, INIT_STATE_INDEX, INIT_STATES_LIST, state_code_to_index, state_index_to_code, action_index_to_code,
                CONSTRAINT_list, C_b_list, N_STATES, N_ACTIONS, ACTIONS_PER_STATE, EPISODE_LENGTH, delta], f)

CONTEXT_VEC_LENGTH = 9
ACTION_CODE_LENGTH = 4
